# Navegación en HTML y Extracción de Tablas con BeautifulSoup

## 1. Introducción a la Estructura HTML

Cuando cargamos una página web en un navegador, lo que realmente obtenemos es un documento **HTML**, que es el código fuente de la página.

### Cómo inspeccionar el HTML de una página web:

1. **Hacer clic derecho en la página → 'Inspeccionar'** (en Chrome o Firefox).

2. **Ver el código fuente** con `Ctrl + U` o `Cmd + U` en Mac.

3. **Escribir en la barra del navegador**: `view-source:https://en.wikipedia.org/wiki/Passengers_of_the_Titanic`.

## 2. Ubicar la Tabla en el HTML

Si inspeccionamos la página de Wikipedia, veremos que los datos de los pasajeros están organizados en **tablas** dentro de elementos `<table>` con la clase `'wikitable sortable'`.

### Ejemplo de estructura HTML de la tabla:

In [ ]:
<table class='wikitable sortable'>
    <tr>
        <th>Name</th>
        <th>Age</th>
        <th>Class</th>
    </tr>
    <tr style='background:#9bddff;'>
        <td>John Smith</td>
        <td>30</td>
        <td>1</td>
    </tr>
</table>

## 3. Cómo Extraer la Tabla con `BeautifulSoup`

Usamos **BeautifulSoup** para encontrar todas las tablas en la página y extraer su contenido.

In [ ]:
import requests
from bs4 import BeautifulSoup

wikipage = "https://en.wikipedia.org/wiki/Passengers_of_the_Titanic"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

response = requests.get(wikipage, headers=headers, timeout=30)
response.raise_for_status()

soup = BeautifulSoup(response.content, "html.parser")

wikitables = soup.find_all("table", class_="wikitable")

print(f"Se encontraron {len(wikitables)} tablas.\n")

for i, table in enumerate(wikitables, start=1):
    header = table.find_previous(["h2", "h3"])
    title = header.get_text(strip=True) if header else "Sin título"
    print(f"Tabla {i}: {title}")

In [ ]:
# Recorremos cada tabla encontrada
for i, table in enumerate(wikitables):
    # Buscamos el encabezado (h2 o h3) más cercano por encima de la tabla
    header = table.find_previous(["h2", "h3"])
    title = header.get_text(strip=True) if header else f"Tabla {i+1} (Sin título)"

    # Contamos filas restando 1 para excluir el encabezado
    rows = table.find_all("tr")
    num_rows = len(rows) - 1

    # Extraemos los nombres de las columnas desde las celdas <th>
    columns = [th.get_text(strip=True) for th in table.find_all("th")]

    print(f"{title}")
    print(f"Número de filas: {num_rows}")
    print(f"Encabezados: {columns}\n")

## 4. Identificar a los Fallecidos en la Tabla (Basado en el Color)

Sabemos que los pasajeros fallecidos tienen `style='background:#9bddff;'`. Entonces, podemos filtrar las filas (`<tr>`) que contienen este atributo.

In [ ]:
# Contamos fallecidos en cada tabla
total_dead = 0
for i, table in enumerate(wikitables[:3]):
    dead_rows = table.find_all("tr", style="background:#9bddff;")
    total_dead += len(dead_rows)
    print(f"Tabla {i+1} ({len(dead_rows)} fallecidos):")

    # Mostramos solo los 3 primeros como ejemplo
    for row in dead_rows[:3]:
        cells = row.find_all("td")
        if len(cells) > 1:
            name = cells[0].get_text(strip=True)
            print(f"  - {name}")
    print()

print(f"Total fallecidos: {total_dead}")

## 5. Construcción de un DataFrame con Pandas

Podemos extraer la información y almacenarla en un **DataFrame de Pandas** para un análisis más profundo.

### Código:

In [ ]:
import pandas as pd

passengers = []

for i, table in enumerate(wikitables[:3]):
    passenger_class = i + 1

    # Leemos los nombres de columna directamente de la tabla
    header_row = table.find("tr")
    col_names = [th.get_text(strip=True) for th in header_row.find_all("th")]

    dead_rows = table.find_all("tr", style="background:#9bddff;")
    all_rows = table.find_all("tr")
    living_rows = [row for row in all_rows if row not in dead_rows]

    for rows, survived in [(living_rows, 1), (dead_rows, 0)]:
        for row in rows:
            cells = row.find_all("td")
            if len(cells) < 2:
                continue

            # zip empareja cada nombre de columna con su celda
            data = {col: cell.get_text(strip=True) for col, cell in zip(col_names, cells)}
            data["class"] = passenger_class
            data["survived"] = survived
            passengers.append(data)

df_passengers = pd.DataFrame(passengers)
print(df_passengers.head(10))
print(f"\nTotal: {len(df_passengers)} pasajeros")
print(f"\nColumnas: {list(df_passengers.columns)}")
print(f"\nPuertos de embarque:\n{df_passengers['Boarded'].value_counts()}")

In [ ]:
df_passengers.tail()

In [ ]:
df_passengers.to_csv("resultado.csv", index=False)

In [ ]:
df_passengers["survived"].value_counts()